Replace the excpet **Dataset.py** block from `gcp2pnet` module

Success in training model

Todo:

- replace parse train_args and using this dateset and gcp2pnet.train.main() to run training.

In [13]:
import os
import random
from scipy import spatial
import networkx as nx
import torch
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
import cv2
import glob
import scipy.io as io
from matplotlib import pyplot as plt

import argparse
import datetime
import random
import time
from pathlib import Path
from IPython.display import clear_output

import torch
from torch.utils.data import DataLoader, DistributedSampler

import os
from tensorboardX import SummaryWriter
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys

sys.path.insert(0, "../")

import gcp2pnet

In [3]:
trimming_size               = 256
padding_size                = 0
learning_rate               = 1e-3 #learning rate for backgrond model
learning_rate_2             = 1e-4 #learning rate for detection head

label_dict = {'Fill': 1, '平べったいけど沈む': 1, '平べったくて浮く': 2, '詰まっているけど浮く': 2, 'Unfill': 2} 
label_type_count = 2+1

ROOT_DIR = r"/home/hwang/d/GoogleDrive/howcanoewang@gmail.com/SharedWithMe/GrainCounting"

training_data_root_dir      = f"{ROOT_DIR}/AnalysisData_small/"
training_data_root_dir2     = f"{ROOT_DIR}/AnalysisData/"
program_files_root_dir      = f"{ROOT_DIR}/programs/"
net_code_dir                = f"{program_files_root_dir}/CrowdCounting-P2PNet"
best_model_file             = f"{net_code_dir}/MultiLevelPyramidFeature_01/best_mae.pth"

####下記フォルダに、学習させたい画像と、Annotationデータをセットで配置します

training_set_root_data_dir                          = training_data_root_dir2 + "SetRootData/"  #こちらの各ディレクトリにjsonと画像ファイルを配置する

training_data_images_dir_before_trimming            = training_data_root_dir + "images/ImageBeforeTrimming/" #こちらに画像を配置する
evaluation_data_images_dir_before_trimming          = training_data_root_dir + "evaluation/ImageBeforeTrimming/" #こちらに画像を配置する
inference_input_path_before_trimming                = training_data_root_dir + "inference/Input/image_before_trimming" #こちらに画像を配置する

training_data_training_label_dir                    = training_data_root_dir + "labels_Training" #こちらにjsonファイルを配置する
training_data_evaluation_label_dir                  = training_data_root_dir + "labels_Evaluation" #こちらにjsonファイルを配置する

####学習用パッチ画像を生成すると更新される
training_data_images_dir                            = training_data_root_dir + "images/" #こちらにパッチ画像が生成される
evaluation_data_images_dir                          = training_data_root_dir + "evaluation/" #こちらにパッチ画像が生成される

training_data_label_dir_before_trimming             = training_data_root_dir + "labels_Training/only_coordinate/ImageBeforeTrimming/" #こちらにjsonを変換したtxtが生成される
evaluation_data_label_dir_before_trimming           = training_data_root_dir + "labels_Evaluation/only_coordinate/ImageBeforeTrimming/" #こちらにjsonを変換したtxtが生成される

training_data_points_dir                            = training_data_root_dir + "labels_Training/only_coordinate/" #こちらにtxtファイルをパッチサイズで切り出したファイルが生成される
evaluation_data_points_dir                          = training_data_root_dir + "labels_Evaluation/only_coordinate/" #こちらにtxtファイルをパッチサイズで切り出したファイルが生成される

training_list_path                                  = training_data_root_dir + "training_list.txt" #こちらに画像⇔txtの対応ファイルが生成される
evaluating_list_path                                = training_data_root_dir + "evaluation_list.txt" #こちらに画像⇔txtの対応ファイルが生成される

#推論用パッチ画像を生成すると更新される
# inference_input_path                                = training_data_root_dir + "inference/Input"#こちらにパッチ画像が生成される
inference_input_path                                = "../data/inference"
# inference_output_path                               = training_data_root_dir + "inference/Output"#こちらに推論結果が生成される
inference_output_path                               = "./outputs/exp_v2"

vis_dir = f"{inference_output_path}/visual_pyramid"
if not os.path.exists(vis_dir):
    os.makedirs(vis_dir)

checkpoints_dir = os.path.abspath("./outputs/exp_v2/models/")
if not os.path.exists(checkpoints_dir):
    os.makedirs(checkpoints_dir)

inference_integrated_image_file_path                = training_data_root_dir + "inference/Output/Integrated"#パッチ統合後画像が生成される

In [4]:
# sys.path.insert(1, net_code_dir)

### Batch codes

In [5]:
###############
# datasets.py
###############

class SHHA(Dataset):
    def __init__(self, data_root, transform=None, train=False, patch=False, flip=False):
        self.root_path = data_root

        #############################
        #   Global variables DANGER #
        #############################
        self.train_lists = training_list_path ## rice grainsの場合
        self.eval_lists = evaluating_list_path ## rice grainsの場合

        # there may exist multiple list files
        # e.g. 
        # ```
        # content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/images/20230209_08_G_a_v05_h11.JPG,/content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/labels_Training/only_coordinate/20230209_08_G_a_v05_h11.txt
        # /content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/images/20221227_07_R_v05_h01.JPG,/content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/labels_Training/only_coordinate/20221227_07_R_v05_h01.txt
        # if train:
        #     self.img_list_file = [name.split(',') for name in open(self.train_lists).read().splitlines()]
        # else:
        #     self.img_list_file = [name.split(',') for name in open(self.eval_lists).read().splitlines()]

        # self.img_list = self.img_list_file

        if train:
            self.img_list = train_img_list
        else:
            self.img_list = eval_img_list

        # number of samples
        self.nSamples = len(self.img_list)

        self.transform = transform
        self.train = train
        self.patch = patch
        self.flip = flip

    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        assert index <= len(self), 'index range error'


        img_path = self.img_list[index][0]
        gt_path = self.img_list[index][1]

        # load image and ground truth
        img, point, labels = load_data((img_path, gt_path), self.train)

        # applu augumentation
        if self.transform is not None:
            img = self.transform(img)

        if self.train:
            # data augmentation -> random scale
            scale_range = [0.5, 1.3]
            min_size = min(img.shape[1:])
            scale = random.uniform(*scale_range)
            # scale the image and points
            if scale * min_size > trimming_size:
                img = torch.nn.functional.upsample_bilinear(img.unsqueeze(0), scale_factor=scale).squeeze(0)
                point *= scale
        # random crop augumentaiton
        if self.train and self.patch:
            img, point,labels = random_crop(img, point, labels)

            for i, _ in enumerate(point):  #03/21 debug
                point[i] = torch.Tensor(point[i])
                labels[i] = torch.Tensor(labels[i])

        # random flipping
        if random.random() > 0.5 and self.train and self.flip:
            # random flip
            img = torch.Tensor(img[:, :, :, ::-1].copy())

            for i, _ in enumerate(point):
                point[i][:, 0] = trimming_size - point[i][:, 0]

        if not self.train:
            point = [point]
            labels = [labels]

        img = torch.Tensor(img)
        # pack up related infos
        target = [{} for i in range(len(point))]

        for i, _ in enumerate(point):  #03/21 debug
            target[i]['point'] = torch.Tensor(point[i])

            if len(labels[0]) > 1:
                target[i]['labels'] = torch.Tensor(labels[i].flatten()).long()
            else:
                target[i]['labels'] = torch.Tensor(labels[i]).long()

            image_id_1 = int(img_path.split('/')[-1].split('.')[0][5:7])
            image_id_1 = torch.Tensor([image_id_1]).long()
            #
            image_id_2 = int(img_path.split('/')[-1].split('.')[0][5:7])
            image_id_2 = torch.Tensor([image_id_2]).long()

            target[i]['image_id_1'] = image_id_1
            target[i]['image_id_2'] = image_id_2

        return img, target


def load_data(img_gt_path, train):
    img_path, gt_path = img_gt_path

    # load the images
    img = cv2.imread(img_path)

    if not img is None:
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # load ground truth points
    points = []
    labels = []
    #
    pts = open(gt_path).read().splitlines()
    for pt_0 in pts:
        pt = eval(pt_0)
        x = float(pt[0])#/2
        y = float(pt[1])#/2
        label = float(pt[2])
        points.append([x, y])
        labels.append([label])

    return img, np.array(points), np.array(labels)


# random crop augumentation
def random_crop(img, den, labels, num_patch=1):
    half_h = trimming_size
    half_w = trimming_size
    result_img = np.zeros([num_patch, img.shape[0], half_h, half_w])
    result_den = []
    result_label = []
    # crop num_patch for each image
    for i in range(num_patch):
        start_h = random.randint(0, img.size(1) - half_h)
        start_w = random.randint(0, img.size(2) - half_w)
        end_h = start_h + half_h
        end_w = start_w + half_w
        # copy the cropped rect
        result_img[i] = img[:, start_h:end_h, start_w:end_w]
        # copy the cropped points
        idx = (den[:, 0] >= start_w) & (den[:, 0] <= end_w) & (den[:, 1] >= start_h) & (den[:, 1] <= end_h)
        # shift the corrdinates
        record_den = den[idx]
        record_label = labels[idx]
        record_den[:, 0] -= start_w
        record_den[:, 1] -= start_h

        result_den.append(record_den)
        result_label.append(record_label)

    return result_img, result_den, result_label

In [6]:
###############
# load_data.py
###############
import torchvision.transforms as standard_transforms

# DeNormalize used to get original images
class DeNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor

def loading_data(data_root):
    # the pre-proccssing transform
    transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]),
    ])
    # create the training dataset
    train_set = SHHA(data_root, train=True, transform=transform, patch=True, flip=True)
    # create the validation dataset
    val_set = SHHA(data_root, train=False, transform=transform)

    return train_set, val_set

In [7]:
from gcp2pnet.utils import DeNormalize, gen_random_scale_n
from gcp2pnet.engine import train_one_epoch, evaluate_crowd_no_overlap

import gcp2pnet.misc as utils

# ############
# # engine.py
# ############
# import math
# import os
# import sys
# from typing import Iterable

# import torch

# sys.path.insert(0, os.path.join (program_files_root_dir,  "CrowdCounting-P2PNet") )
# import util.misc as utils
# from util.misc import NestedTensor
# # this util is from previous P2PNet directory

# import numpy as np
# import time
# import torchvision.transforms as standard_transforms
# import cv2

# class DeNormalize(object):
#     def __init__(self, mean, std):
#         self.mean = mean
#         self.std = std

#     def __call__(self, tensor):
#         for t, m, s in zip(tensor, self.mean, self.std):
#             t.mul_(s).add_(m)
#         return tensor
# #
# def gen_random_scale_n(img, rnd=3):
#     np.random.seed(rnd)
#     scale = torch.tensor(np.random.uniform(0.1, 1.91, (1,1, img.size()[2],img.size()[3]))).float()
#     return torch.mul(img, scale)
# ##
# def vis(samples, targets, pred, vis_dir, epoch, predict_cnt, gt_cnt):
#     '''
#     samples -> tensor: [batch, 3, H, W]
#     targets -> list of dict: [{'points':[], 'image_id': str}]
#     pred -> list: [num_preds, 2]
#     '''
#     gts = [t['point'].tolist() for t in targets]

#     pil_to_tensor = standard_transforms.ToTensor()

#     restore_transform = standard_transforms.Compose([
#         DeNormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         standard_transforms.ToPILImage()
#     ])
#     # draw one by one
#     for idx in range(samples.shape[0]):
#         sample = restore_transform(samples[idx])
#         sample = pil_to_tensor(sample.convert('RGB')).numpy() * 255
#         sample_gt = sample.transpose([1, 2, 0])[:, :, ::-1].astype(np.uint8).copy()
#         sample_pred = sample.transpose([1, 2, 0])[:, :, ::-1].astype(np.uint8).copy()

#         max_len = np.max(sample_gt.shape)

#         size = 5
#         # draw gt
#         for t in gts[idx]:
#             sample_gt = cv2.circle(sample_gt, (int(t[0]), int(t[1])), size, (0, 255, 0), -1)
#         # draw predictions
#         for p in pred[idx]:
#             sample_pred = cv2.circle(sample_pred, (int(p[0]), int(p[1])), size, (0, 0, 255), -1)

#         name_1 = targets[idx]['image_id_1']
#         name_2 = targets[idx]['image_id_2']
#         #################
#         fig = plt.figure()
#         ax1 = fig.add_subplot(1, 2, 1)
#         ax1.imshow(sample_gt)
#         ax1.get_xaxis().set_visible(False)
#         ax1.get_yaxis().set_visible(False)
#         ax2 = fig.add_subplot(1, 2, 2)
#         ax2.imshow(sample_pred)
#         ax2.get_xaxis().set_visible(False)
#         ax2.get_yaxis().set_visible(False)
#         fig.suptitle('manual count=%4.2f, inferred count=%4.2f'%(gt_cnt, predict_cnt), fontsize=10)
#         plt.tight_layout(rect=[0, 0, 0.95, 0.95]) # maize tassels counting
#         plt.savefig(os.path.join(vis_dir, '{}_{}_id_{}_ind_{}.jpg'.format(epoch, idx, int(name_1), int(name_2))), bbox_inches='tight', dpi = 300)
#         plt.close()


# # the training routine
# def train_one_epoch(model: torch.nn.Module, criterion: torch.nn.Module,
#                     data_loader: Iterable, optimizer: torch.optim.Optimizer,
#                     device: torch.device, epoch: int, max_norm: float = 0):
#     model.train()
#     criterion.train()
#     metric_logger = utils.MetricLogger(delimiter="  ")
#     metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
#     # iterate all training samples
#     for samples, targets in data_loader:
#         #print("samples {}".format(samples.size()))
#         #samples = gen_random_scale_n(samples)
#         #
#         samples = samples.to(device)
#         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#         # forward
#         outputs = model(samples)
#         #
# #        print("outputs")
# #        print(outputs)


# #        print("targets")
# #        print(targets.shape)


#         # calc the losses
#         loss_dict = criterion(outputs, targets)
#         weight_dict = criterion.weight_dict
#         losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

#         # reduce all losses (get the mean values)
#         loss_dict_reduced = utils.reduce_dict(loss_dict)
#         loss_dict_reduced_unscaled = {f'{k}_unscaled': v
#                                       for k, v in loss_dict_reduced.items()}
#         loss_dict_reduced_scaled = {k: v * weight_dict[k]
#                                     for k, v in loss_dict_reduced.items() if k in weight_dict}
#         losses_reduced_scaled = sum(loss_dict_reduced_scaled.values())

#         loss_value = losses_reduced_scaled.item()

#         if not math.isfinite(loss_value):
#             print("Loss is {}, stopping training".format(loss_value))
#             print(loss_dict_reduced)
#             sys.exit(1)
#         # backward
#         optimizer.zero_grad()
#         losses.backward()
#         if max_norm > 0:
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
#         optimizer.step()
#         # update logger
#         metric_logger.update(loss=loss_value, **loss_dict_reduced_scaled, **loss_dict_reduced_unscaled)
#         metric_logger.update(lr=optimizer.param_groups[0]["lr"])
#     # gather the stats from all processes
#     metric_logger.synchronize_between_processes()
#     print("Averaged stats:", metric_logger)
#     return {k: meter.global_avg for k, meter in metric_logger.meters.items()}






# # the inference routine
# @torch.no_grad()
# def evaluate_crowd_no_overlap(model, data_loader, device, epoch, threshold, vis_dir=None):
#     model.eval()

#     metric_logger = utils.MetricLogger(delimiter="  ")
#     metric_logger.add_meter('class_error', utils.SmoothedValue(window_size=1, fmt='{value:.2f}'))
#     # run inference on all images to calc MAE
#     maes = []
#     mses = []
#     for samples, targets in data_loader:

#         gt_cnt = targets[0]['point'].shape[0]

#         gt_cnt_0 = 0
#         gt_cnt_1 = 0

#         for label in targets[0]['labels']:
#           if label == 1:
#             gt_cnt_0 += 1
#           elif label == 2:
#             gt_cnt_1 += 1

# ########################

# #        samples = torch.Tensor(samples).unsqueeze(0)
#         samples = samples.to(device)
#         # run inference
#         outputs = model(samples)

#         outputs_points = outputs['pred_points'][0]

#         outputs_scores = []
#         for i in range(label_type_count - 1):
#           outputs_scores.append(torch.nn.functional.softmax(outputs['pred_logits'], -1)[:, :, i + 1][0])



#         # filter the predictions
#         points_n_for_all_class = []
#         scores_n_for_all_class = []

#         count_0_for_all_points = []
#         count_1_for_all_points = []


#         for i in range(label_type_count - 1):

#           points = outputs_points   [outputs_scores[i] > threshold].detach().cpu().numpy()#.tolist()
#           scores = outputs_scores[i][outputs_scores[i] > threshold].detach().cpu().numpy()#.tolist()

#           if points.shape[0]<10000 and points.shape[0] != 0:
#               cutoff = 500/points.shape[0]
#               if cutoff<20:
#                   cutoff = 20
#               components = nx.connected_components(
#                   nx.from_edgelist(
#                       (i, j) for i, js in enumerate(
#                           spatial.KDTree(points).query_ball_point(points, cutoff)
#                       )
#                       for j in js
#                   )
#               )


#               clusters = {j: i for i, js in enumerate(components) for j in js}


#               # reorganize the points to the order of clusters
#               points_reo = np.zeros(points.shape)
#               scores_reo = np.zeros(scores.shape)

#               i = 0
#               for key in clusters.keys():
#                   points_reo[i,:] = points[key,:]
#                   scores_reo[i] = scores[key]
#                   i+=1


#               # points_n has the same order as clusters
#               res = [clusters[key] for key in clusters.keys()]
#               res_n = np.array(res).reshape(-1,1)


#               points_n = []
#               scores_n = []
#               for i in np.unique(res_n):

#                   tmp_points = points_reo[np.where(res_n[:,0] == i)]
#                   tmp_scores = scores_reo[np.where(res_n[:,0] == i)]

#                   points_n.append( [np.mean(tmp_points[:,0]), np.mean(tmp_points[:,1])])
#                   scores_n.append( [np.mean(tmp_scores[:])])
# #                    scores_n.append( [np.amax(tmp_scores[:])])

#           else:
#               points_n = points.tolist()
#               scores_n = scores.tolist()

#           points_n_for_all_class.append(points_n)
#           scores_n_for_all_class.append(scores_n)


#         #推論結果の点群を走査して、場所が重複するものがあれば、第三のカテゴリに変更する。(縦も横も7ピクセル以内かどうか)

#         point_box = []
#         score_box = []
#         estimated_result = []


#         for i, point_set in enumerate(points_n_for_all_class):
#           for p in point_set:
#               if i == 0:
#                 point_box.append([p,0])
#               if i == 1:
#                 point_box.append([p,1])

#         for i, score_set in enumerate(scores_n_for_all_class):
#           for s in score_set:
#               if i == 0:
#                 score_box.append([s,0])
#               if i == 1:
#                 score_box.append([s,1])


#         prox_distance = 25  #これより近い距離に別の点があれば、同じ点であるとみなして統合します。

#         data_for_print = []

#         count_0 = 0
#         count_1 = 0

#         #distance以内に、自分よりscoreが高い点が無ければ、valid_flagが１のまま残り、その点をカウントし、画像にプロットする。
#         for i, test_point in enumerate(point_box):

#           valid_flag = 1

#           x_test_point = test_point[0][0]
#           y_test_point = test_point[0][1]
#           score_test   = score_box[i][0][0]
#           class_test   = score_box[i][1]

#           for j, compared_point in enumerate(point_box):
#             x_compared_point = compared_point[0][0]
#             y_compared_point = compared_point[0][1]
#             score_compared   = score_box[j][0][0]
#             dist = math.sqrt((x_test_point - x_compared_point) * (x_test_point - x_compared_point) + (y_test_point - y_compared_point) * (y_test_point - y_compared_point))

#             if ((dist < prox_distance) and (score_test < score_compared)):
#               valid_flag = 0

#           if valid_flag == 1:
#             if class_test == 0:
#               count_0 = count_0 + 1
#             else:
#               count_1 = count_1 + 1


# #########################
#       # accumulate MAE, MSE
#         mae = abs(count_0 - gt_cnt_0) + abs(count_1 - gt_cnt_1)
#         mse = (count_0 - gt_cnt_0) * (count_0 - gt_cnt_0) + (count_1 - gt_cnt_1) * (count_1 - gt_cnt_1)

# #        print("count_0:" +  str(count_0) + "  count_1:" + str(count_1) + "  gt_cnt_0:" + str(gt_cnt_0) + "  gt_cnt_1:" + str(gt_cnt_1)+ "  mae:" + str(mae)+ "  mse:" + str(mse))

#         maes.append(float(mae))
#         mses.append(float(mse))
#     # calc MAE, MSE
#     mae = np.mean(maes)
#     mse = np.sqrt(np.mean(mses))

#     return mae, mse

### model definition

In [8]:
from gcp2pnet.models.attention import SpatialAttention, ChannelwiseAttention

# import argparse
# import datetime
# import random
# import time
# from pathlib import Path
# from IPython.display import clear_output

# import torch
# from torch.utils.data import DataLoader, DistributedSampler

# import os
# from tensorboardX import SummaryWriter
# import warnings
# warnings.filterwarnings('ignore')

# ## attention module
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# #
# class SpatialAttention(nn.Module):
#     def __init__(self, in_channels, kernel_size=9):
#         super(SpatialAttention, self).__init__()

#         self.kernel_size = kernel_size
#         self.in_channels = in_channels
#         pad = (self.kernel_size-1)//2  # Padding on one side for stride 1

#         self.grp1_conv1k = nn.Conv2d(self.in_channels, self.in_channels//2, (1, self.kernel_size), padding=(0, pad))
#         self.grp1_bn1 = nn.BatchNorm2d(self.in_channels//2)
#         self.grp1_convk1 = nn.Conv2d(self.in_channels//2, 1, (self.kernel_size, 1), padding=(pad, 0))
#         self.grp1_bn2 = nn.BatchNorm2d(1)

#         self.grp2_convk1 = nn.Conv2d(self.in_channels, self.in_channels//2, (self.kernel_size, 1), padding=(pad, 0))
#         self.grp2_bn1 = nn.BatchNorm2d(self.in_channels//2)
#         self.grp2_conv1k = nn.Conv2d(self.in_channels//2, 1, (1, self.kernel_size), padding=(0, pad))
#         self.grp2_bn2 = nn.BatchNorm2d(1)

#     def forward(self, input_):
#         # Generate Group 1 Features
#         grp1_feats = self.grp1_conv1k(input_)
#         grp1_feats = F.relu(self.grp1_bn1(grp1_feats))
#         grp1_feats = self.grp1_convk1(grp1_feats)
#         grp1_feats = F.relu(self.grp1_bn2(grp1_feats))

#         # Generate Group 2 features
#         grp2_feats = self.grp2_convk1(input_)
#         grp2_feats = F.relu(self.grp2_bn1(grp2_feats))
#         grp2_feats = self.grp2_conv1k(grp2_feats)
#         grp2_feats = F.relu(self.grp2_bn2(grp2_feats))

#         added_feats = torch.sigmoid(torch.add(grp1_feats, grp2_feats))
#         added_feats = added_feats.expand_as(input_).clone()

#         return added_feats


# class ChannelwiseAttention(nn.Module):
#     def __init__(self, in_channels):
#         super(ChannelwiseAttention, self).__init__()

#         self.in_channels = in_channels

#         self.linear_1 = nn.Linear(self.in_channels, self.in_channels//4)
#         self.linear_2 = nn.Linear(self.in_channels//4, self.in_channels)

#     def forward(self, input_):
#         n_b, n_c, h, w = input_.size()

#         feats = F.adaptive_avg_pool2d(input_, (1, 1)).view((n_b, n_c))
#         feats = F.relu(self.linear_1(feats))
#         feats = torch.sigmoid(self.linear_2(feats))

#         # Activity regularizer
#         ca_act_reg = torch.mean(feats)

#         feats = feats.view((n_b, n_c, 1, 1))
#         feats = feats.expand_as(input_).clone()

#         return feats, ca_act_reg
# ## note on the output: attention modules doesn't af

In [9]:
from gcp2pnet.models.pyrimid import (
    vgg_conv1_2, vgg_conv2_2, vgg_conv3_3, vgg_conv4_3, vgg_conv5_3,
    conv_1_2_hook, conv_2_2_hook, conv_3_3_hook, conv_4_3_hook, conv_5_3_hook,
    CPFE, SODModel, 
)

# ####                    pyramid feature attention. (here you adjust different feature levels and attentions)
# import numpy as np
# import cv2
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torchvision.models as models


# vgg_conv1_2 = vgg_conv2_2 = vgg_conv3_3 = vgg_conv4_3 = vgg_conv5_3 = None


# def conv_1_2_hook(module, input, output):
#     global vgg_conv1_2
#     vgg_conv1_2 = output
#     return None

# def conv_2_2_hook(module, input, output):
#     global vgg_conv2_2
#     vgg_conv2_2 = output
#     return None

# def conv_3_3_hook(module, input, output):
#     global vgg_conv3_3
#     vgg_conv3_3 = output
#     return None

# def conv_4_3_hook(module, input, output):
#     global vgg_conv4_3
#     vgg_conv4_3 = output
#     return None

# def conv_5_3_hook(module, input, output):
#     global vgg_conv5_3
#     vgg_conv5_3 = output
#     return None

# class CPFE(nn.Module):
#     def __init__(self, feature_layer=None, out_channels=8):
#         super(CPFE, self).__init__()

#         self.dil_rates = [3, 5, 7]

#         # Determine number of in_channels from VGG-16 feature layer
#         if feature_layer == 'conv5_3':
#             self.in_channels = 512
#         elif feature_layer == 'conv4_3':
#             self.in_channels = 512
#         elif feature_layer == 'conv3_3':
#             self.in_channels = 256

#         # Define layers
#         self.conv_1_1 = nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=1, bias=False)
#         self.conv_dil_3 = nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=3,
#                                     stride=1, dilation=self.dil_rates[0], padding=self.dil_rates[0], bias=False)
#         self.conv_dil_5 = nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=3,
#                                     stride=1, dilation=self.dil_rates[1], padding=self.dil_rates[1], bias=False)
#         self.conv_dil_7 = nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=3,
#                                     stride=1, dilation=self.dil_rates[2], padding=self.dil_rates[2], bias=False)

#         self.bn = nn.BatchNorm2d(out_channels*4)

#     def forward(self, input_):
#         # Extract features
#         conv_1_1_feats = self.conv_1_1(input_)
#         conv_dil_3_feats = self.conv_dil_3(input_)
#         conv_dil_5_feats = self.conv_dil_5(input_)
#         conv_dil_7_feats = self.conv_dil_7(input_)

#         # Aggregate features
#         concat_feats = torch.cat((conv_1_1_feats, conv_dil_3_feats, conv_dil_5_feats, conv_dil_7_feats), dim=1)
#         bn_feats = F.relu(self.bn(concat_feats))

#         return bn_feats

# class SODModel(nn.Module):
#     def __init__(self):
#         super(SODModel, self).__init__()

#         # Load the [partial] VGG-16 model
#         self.vgg16 = models.vgg16(pretrained=True).features

#         # Extract and register intermediate features of VGG-16
#         self.vgg16[3].register_forward_hook(conv_1_2_hook)
#         self.vgg16[8].register_forward_hook(conv_2_2_hook)
#         self.vgg16[15].register_forward_hook(conv_3_3_hook)
#         self.vgg16[22].register_forward_hook(conv_4_3_hook)
#         self.vgg16[29].register_forward_hook(conv_5_3_hook)

#         # Initialize layers for high level (hl) feature (conv3_3, conv4_3, conv5_3) processing
#         self.cpfe_conv3_3 = CPFE(feature_layer='conv3_3')
#         self.cpfe_conv4_3 = CPFE(feature_layer='conv4_3')
#         self.cpfe_conv5_3 = CPFE(feature_layer='conv5_3')
#         #  channel attention, remove if not needed
#         # self.cha_att = ChannelwiseAttention(in_channels=96)  # in_channels = 3 x (8 x 4)

#         self.hl_conv1 = nn.Conv2d(96, 8, (3, 3), padding=1)
#         self.hl_bn1 = nn.BatchNorm2d(8)

#         # Initialize layers for low level (ll) feature (conv1_2 and conv2_2) processing
#         #self.ll_conv_0 = nn.Conv2d(3, 8, (3, 3), padding=1)
#         #self.ll_bn_0 = nn.BatchNorm2d(8)
#         self.ll_conv_1 = nn.Conv2d(64, 8, (3, 3), padding=1)
#         self.ll_bn_1 = nn.BatchNorm2d(8)
#         self.ll_conv_2 = nn.Conv2d(128, 8, (3, 3), padding=1)
#         self.ll_bn_2 = nn.BatchNorm2d(8)
#         self.ll_conv_3 = nn.Conv2d(19, 8, (3, 3), padding=1)
#         self.ll_bn_3 = nn.BatchNorm2d(8)

#         self.spa_att = SpatialAttention(in_channels=8)

#         # Initialize layers for fused features (ff) processing
#         #self.ff_conv_1 = nn.Conv2d(16, 1, (3, 3), padding=1)

#     def forward(self, input_):
#         global vgg_conv1_2, vgg_conv2_2, vgg_conv3_3, vgg_conv4_3, vgg_conv5_3

#         # Pass input_ through vgg16 to generate intermediate features
#         self.vgg16(input_)
#         # print(vgg_conv1_2.size())
#         # print(vgg_conv2_2.size())
#         # print(vgg_conv3_3.size())
#         # print(vgg_conv4_3.size())
#         # print(vgg_conv5_3.size())

#         # Process high level features
#         conv3_cpfe_feats = self.cpfe_conv3_3(vgg_conv3_3)
#         conv4_cpfe_feats = self.cpfe_conv4_3(vgg_conv4_3)
#         conv5_cpfe_feats = self.cpfe_conv5_3(vgg_conv5_3)

#         conv4_cpfe_feats = F.interpolate(conv4_cpfe_feats, scale_factor=2, mode='bilinear', align_corners=True) # reduce spatial dimension by 2
#         conv5_cpfe_feats = F.interpolate(conv5_cpfe_feats, scale_factor=4, mode='bilinear', align_corners=True)

#         conv_345_feats = torch.cat((conv3_cpfe_feats, conv4_cpfe_feats, conv5_cpfe_feats), dim=1)

#         # 11,03,2022, remove channel attention
#         #conv_345_ca, ca_act_reg = self.cha_att(conv_345_feats)
#         #conv_345_feats = torch.mul(conv_345_feats, conv_345_ca)

#         conv_345_feats = self.hl_conv1(conv_345_feats)
#         conv_345_feats = F.relu(self.hl_bn1(conv_345_feats))
#         #conv_345_feats = F.interpolate(conv_345_feats, scale_factor=2, mode='bilinear', align_corners=True) # to increase the spatial resolution by 2 (is removed now)

#         # Process low level features
#         conv0_feats = input_ #add original image as low feature level input
#         #conv0_feats = F.relu(self.ll_bn_0(conv0_feats))
#         conv1_feats = self.ll_conv_1(vgg_conv1_2)
#         conv1_feats = F.relu(self.ll_bn_1(conv1_feats))
#         conv2_feats = self.ll_conv_2(vgg_conv2_2)
#         conv2_feats = F.relu(self.ll_bn_2(conv2_feats))

#         conv0_feats = F.interpolate(conv0_feats, scale_factor=0.25, mode='bilinear', align_corners=True)
#         conv1_feats = F.interpolate(conv1_feats, scale_factor=0.25, mode='bilinear', align_corners=True)
#         conv2_feats = F.interpolate(conv2_feats, scale_factor=0.5, mode='bilinear', align_corners=True)

#         conv_12_feats = torch.cat((conv0_feats, conv1_feats, conv2_feats), dim=1)
#         conv_12_feats = self.ll_conv_3(conv_12_feats)
#         conv_12_feats = F.relu(self.ll_bn_3(conv_12_feats))

#         conv_12_sa = self.spa_att(conv_12_feats)
#         conv_12_feats = torch.mul(conv_12_feats, conv_12_sa)

#         # Fused features
#         fused_features_l = torch.sigmoid(conv_12_feats)
#         fused_features_h = torch.sigmoid(conv_345_feats)
#         #fused_feats_l = conv_12_feats
#         #fused_features_lh = torch.add(conv_12_feats, conv_345_feats)
#         #fused_features_lh = torch.sigmoid(fused_features_lh)
#         #
#         #fused_feats_h = conv_345_feats
#         #fused_feats_h = torch.sigmoid(fused_feats_h)

#         return fused_features_l, fused_features_h #, fused_feats_h


In [10]:
from gcp2pnet.models.matcher import HungarianMatcher_Crowd
'''
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
Mostly copy-paste from DETR (https://github.com/facebookresearch/detr).

import torch
from scipy.optimize import linear_sum_assignment
from torch import nn


class HungarianMatcher_Crowd(nn.Module):
    """This class computes an assignment between the targets and the predictions of the network

    For efficiency reasons, the targets don't include the no_object. Because of this, in general,
    there are more predictions than targets. In this case, we do a 1-to-1 matching of the best predictions,
    while the others are un-matched (and thus treated as non-objects).
    """

    def __init__(self, cost_class: float = 1, cost_point: float = 1):
        """Creates the matcher

        Params:
            cost_class: This is the relative weight of the foreground object
            cost_point: This is the relative weight of the L1 error of the points coordinates in the matching cost
        """
        super().__init__()
        self.cost_class = cost_class
        self.cost_point = cost_point
        assert cost_class != 0 or cost_point != 0, "all costs cant be 0"

    @torch.no_grad()
    def forward(self, outputs, targets):
        """ Performs the matching

        Params:
            outputs: This is a dict that contains at least these entries:
                 "pred_logits": Tensor of dim [batch_size, num_queries, num_classes] with the classification logits
                 "points": Tensor of dim [batch_size, num_queries, 2] with the predicted point coordinates

            targets: This is a list of targets (len(targets) = batch_size), where each target is a dict containing:
                 "labels": Tensor of dim [num_target_points] (where num_target_points is the number of ground-truth
                           objects in the target) containing the class labels
                 "points": Tensor of dim [num_target_points, 2] containing the target point coordinates

        Returns:
            A list of size batch_size, containing tuples of (index_i, index_j) where:
                - index_i is the indices of the selected predictions (in order)
                - index_j is the indices of the corresponding selected targets (in order)
            For each batch element, it holds:
                len(index_i) = len(index_j) = min(num_queries, num_target_points)
        """
        bs, num_queries = outputs["pred_logits"].shape[:2]

        # We flatten to compute the cost matrices in a batch
        out_prob = outputs["pred_logits"].flatten(0, 1).softmax(-1)  # [batch_size * num_queries, num_classes]
        out_points = outputs["pred_points"].flatten(0, 1)  # [batch_size * num_queries, 2]

        # Also concat the target labels and points
        # tgt_ids = torch.cat([v["labels"] for v in targets])
        tgt_ids = torch.cat([v["labels"] for v in targets])
        tgt_points = torch.cat([v["point"] for v in targets])


#03/21 debug        print("target:")
#03/21 debug        print(targets)
#03/21 debug        print("target_ids:")
#03/21 debug        print(tgt_ids)
#03/21 debug        print("target_points:")
#03/21 debug        print(tgt_points)

#03/21 debug        print(self.cost_point)
#03/21 debug        print(self.cost_class)


        # Compute the classification cost. Contrary to the loss, we don't use the NLL,
        # but approximate it in 1 - proba[target class].
        # The 1 is a constant that doesn't change the matching, it can be ommitted.
        cost_class = -out_prob[:, tgt_ids]

        # Compute the L2 cost between point
#03/21 debug        print("out_points:")
#03/21 debug        print(out_points)
#03/21 debug        print("tgt_points:")
#03/21 debug        print(tgt_points)
        cost_point = torch.cdist(out_points, tgt_points, p=2)

        # Compute the giou cost between point

        # Final cost matrix
#03/21 debug        print("cost_point:")
#03/21 debug        print(cost_point.shape)
#03/21 debug        print("cost_class:")
#03/21 debug        print(cost_class.shape)
#03/21 debug        print(cost_class.squeeze().shape)
#03/21 debug        print("self.cost_point:")
#03/21 debug        print(self.cost_point)
#03/21 debug        print("self.cost_class:")
#03/21 debug        print(self.cost_class)


#3/25 debug        C = self.cost_point * cost_point + self.cost_class * cost_class.squeeze()
        if cost_class.dim() == 3:
          cost_class = torch.squeeze(cost_class, 2)

        C = self.cost_point * cost_point + self.cost_class * cost_class
        C = C.view(bs, num_queries, -1).cpu()

        sizes = [len(v["point"]) for v in targets]


#03/21 debug        print("sizes")
#03/21 debug        print(sizes[0])
#03/21 debug        print(np.array(sizes).shape)



#03/21 debug        print("C")
#03/21 debug        print(np.array(C).shape)

        indices = [linear_sum_assignment(c[i]) for i, c in enumerate(C.split(sizes, -1))]

#03/21 debug                indices = [linear_sum_assignment(c[i]) for i, c in enumerate(C.split(sizes, -1))]
        return [(torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64)) for i, j in indices]


def build_matcher_crowd(args):
    return HungarianMatcher_Crowd(cost_class=args.set_cost_class, cost_point=args.set_cost_point)
'''

'\n# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved\nMostly copy-paste from DETR (https://github.com/facebookresearch/detr).\n\nimport torch\nfrom scipy.optimize import linear_sum_assignment\nfrom torch import nn\n\n\nclass HungarianMatcher_Crowd(nn.Module):\n    """This class computes an assignment between the targets and the predictions of the network\n\n    For efficiency reasons, the targets don\'t include the no_object. Because of this, in general,\n    there are more predictions than targets. In this case, we do a 1-to-1 matching of the best predictions,\n    while the others are un-matched (and thus treated as non-objects).\n    """\n\n    def __init__(self, cost_class: float = 1, cost_point: float = 1):\n        """Creates the matcher\n\n        Params:\n            cost_class: This is the relative weight of the foreground object\n            cost_point: This is the relative weight of the L1 error of the points coordinates in the matching cost\n        """

In [11]:
from gcp2pnet.models.p2pnet import (
    RegressionModel, ClassificationModel, 
    generate_anchor_points, shift, 
    AnchorPoints, P2PNet, SetCriterion_Crowd, 
    build, build_eval,
)

'''
############
# p2pnet.py
############

# build model p2pNet.py
import torch
import torch.nn.functional as F
from torch import nn

from util.misc import (NestedTensor, nested_tensor_from_tensor_list,
                       accuracy, get_world_size, interpolate,
                       is_dist_avail_and_initialized)

# from models.backbone import build_backbone
# from models.matcher import build_matcher_crowd

import numpy as np
import time

# the network frmawork of the regression branch
class RegressionModel(nn.Module):
    def __init__(self, num_features_in, num_anchor_points=4, feature_size=32):
        super(RegressionModel, self).__init__()

        self.conv1 = nn.Conv2d(num_features_in, feature_size, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()

        self.conv2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()

        self.output = nn.Conv2d(feature_size, num_anchor_points * 2, kernel_size=3, padding=1) # one point has two coordinates
    # sub-branch forward
    def forward(self, x):
        out = self.conv1(x)
        out = self.act1(out)

        out = self.conv2(out)
        out = self.act2(out)

        out = self.output(out)

        out = out.permute(0, 2, 3, 1)

        return out.contiguous().view(out.shape[0], -1, 2)

# the network frmawork of the classification branch
class ClassificationModel(nn.Module):
    def __init__(self, num_features_in, num_anchor_points=4, num_classes=80, prior=0.01, feature_size=32):
        super(ClassificationModel, self).__init__()

        self.num_classes = num_classes
        self.num_anchor_points = num_anchor_points

        self.conv1 = nn.Conv2d(num_features_in, feature_size, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()

        self.conv2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()

        self.output = nn.Conv2d(feature_size, num_anchor_points * num_classes, kernel_size=3, padding=1) # one classes, only positives
        self.output_act = nn.Sigmoid()
    # sub-branch forward
    def forward(self, x):
        out = self.conv1(x)
        out = self.act1(out)

        out = self.conv2(out)
        out = self.act2(out)

        out = self.output(out)

        out1 = out.permute(0, 2, 3, 1)

        batch_size, width, height, _ = out1.shape

        out2 = out1.view(batch_size, width, height, self.num_anchor_points, self.num_classes)

        return out2.contiguous().view(x.shape[0], -1, self.num_classes)

# generate the reference points in grid layout
def generate_anchor_points(stride=8, row=3, line=3):
    row_step = stride / row
    line_step = stride / line

    shift_x = (np.arange(1, line + 1) - 0.5) * line_step - stride / 2
    shift_y = (np.arange(1, row + 1) - 0.5) * row_step - stride / 2

    shift_x, shift_y = np.meshgrid(shift_x, shift_y)

    anchor_points = np.vstack((
        shift_x.ravel(), shift_y.ravel()
    )).transpose()

    return anchor_points
# shift the meta-anchor to get an acnhor points
def shift(shape, stride, anchor_points):
    shift_x = (np.arange(0, shape[1]) + 0.5)* stride
    shift_y = (np.arange(0, shape[0]) + 0.5)* stride

    shift_x, shift_y = np.meshgrid(shift_x, shift_y)

    shifts = np.vstack((
        shift_x.ravel(), shift_y.ravel()
    )).transpose()

    A = anchor_points.shape[0]
    K = shifts.shape[0]
    all_anchor_points = (anchor_points.reshape((1, A, 2)) + shifts.reshape((1, K, 2)).transpose((1, 0, 2)))
    all_anchor_points = all_anchor_points.reshape((K * A, 2))

    return all_anchor_points

# this class generate all reference points on all pyramid levels
class AnchorPoints(nn.Module):
    def __init__(self, pyramid_levels=None, strides=None, row=3, line=3):
        super(AnchorPoints, self).__init__()

        if pyramid_levels is None:
            self.pyramid_levels = [3, 4, 5, 6, 7]
        else:
            self.pyramid_levels = pyramid_levels

        if strides is None:
            self.strides = [2 ** x for x in self.pyramid_levels]

        self.row = row
        self.line = line

    def forward(self, image):
        image_shape = image.shape[2:]
        image_shape = np.array(image_shape)
        image_shapes = [(image_shape + 2 ** x - 1) // (2 ** x) for x in self.pyramid_levels] # calcualtes the output size of the model (image of 128*128 to feature map of 16*16)

        all_anchor_points = np.zeros((0, 2)).astype(np.float32)
        # get reference points for each level
        for idx, p in enumerate(self.pyramid_levels):
            anchor_points = generate_anchor_points(2**p, row=self.row, line=self.line)
            shifted_anchor_points = shift(image_shapes[idx], self.strides[idx], anchor_points)
            all_anchor_points = np.append(all_anchor_points, shifted_anchor_points, axis=0)

        all_anchor_points = np.expand_dims(all_anchor_points, axis=0)
        # send reference points to device
        if torch.cuda.is_available():
            return torch.from_numpy(all_anchor_points.astype(np.float32)).cuda()
        else:
            return torch.from_numpy(all_anchor_points.astype(np.float32))
##
# the defenition of the P2PNet model
class P2PNet(nn.Module):
    def __init__(self, row=2, line=2):
        super().__init__()
        self.num_classes = label_type_count
#        self.num_classes = label_type_count + 1
        # the number of all anchor points
        num_anchor_points = row * line

        self.regression = RegressionModel(num_features_in=8, num_anchor_points=num_anchor_points)
        self.classification = ClassificationModel(num_features_in=8, \
                                            num_classes=self.num_classes, \
                                            num_anchor_points=num_anchor_points)

        self.anchor_points = AnchorPoints(pyramid_levels=[2,], row=row, line=line) # remember to change pyramid level when you change feature input

        #self.fpn = Decoder_stg3(128, 256, 512, 512)
        self.fpn = SODModel()

    def forward(self, samples: NestedTensor):
        # get the backbone features
        # forward the feature pyramid
        features_fpn_l, features_fpn_h = self.fpn(samples) # output = bach_size, channel, Height, Weight

        batch_size = features_fpn_l.size()[0]
        #print("features_fpn_1 size")
        #print(features_fpn_1.size())
        #print(features_fpn[1].size())
        #print(features_fpn[2].size())
        #  put low level features for points regression
        regression = self.regression(features_fpn_l) * 100 # 8x
        #  put high level features for classification
        classification = self.classification(features_fpn_h)
        # generate reference points
        anchor_points = self.anchor_points(samples).repeat(batch_size, 1, 1)
        # decode the points as prediction
        #print("regression {}".format(regression.size()))
        #print("anchor_points {}".format(anchor_points.size()))
        output_coord = regression + anchor_points
        output_class = classification
        out = {'pred_logits': output_class, 'pred_points': output_coord}

        return out

class SetCriterion_Crowd(nn.Module):

    def __init__(self, num_classes, matcher, weight_dict, eos_coef, losses):
        """ Create the criterion.
        Parameters:
            num_classes: number of object categories, omitting the special no-object category
            matcher: module able to compute a matching between targets and proposals
            weight_dict: dict containing as key the names of the losses and as values their relative weight.
            eos_coef: relative classification weight applied to the no-object category
            losses: list of all the losses to be applied. See get_loss for list of available losses.
        """
        super().__init__()
        self.num_classes = num_classes
        self.matcher = matcher
        self.weight_dict = weight_dict
        self.eos_coef = eos_coef
        self.losses = losses
#        empty_weight = torch.ones(self.num_classes + 1)  03/19バグ修正
        empty_weight = torch.ones(self.num_classes)
        empty_weight[0] = self.eos_coef
        self.register_buffer('empty_weight', empty_weight)

#03/21 debug        print("weight_vector:")
#        print(empty_weight.())

    def loss_labels(self, outputs, targets, indices, num_points):
        """Classification loss (NLL)
        targets dicts must contain the key "labels" containing a tensor of dim [nb_target_boxes]
        """
        assert 'pred_logits' in outputs
        src_logits = outputs['pred_logits']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t["labels"][J] for t, (_, J) in zip(targets, indices)])
        target_classes = torch.full(src_logits.shape[:2], 0,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o.squeeze()

        torch.set_printoptions(edgeitems=1000)
#03/21 debug        print("logits")
#03/21 debug        print(src_logits.transpose(1, 2).size())
#03/21 debug        print("target_class:")
#03/21 debug        print(target_classes.size())
#03/21 debug        print("weight:")
#03/21 debug        print(self.empty_weight.size())

        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.empty_weight)
        losses = {'loss_ce': loss_ce}

        return losses

    def loss_points(self, outputs, targets, indices, num_points):

        assert 'pred_points' in outputs
        idx = self._get_src_permutation_idx(indices)
        src_points = outputs['pred_points'][idx]
        target_points = torch.cat([t['point'][i] for t, (_, i) in zip(targets, indices)], dim=0)
        loss_bbox = F.mse_loss(src_points, target_points, reduction='none')

        losses = {}
        losses['loss_points'] = loss_bbox.sum() / num_points

        return losses

    def _get_src_permutation_idx(self, indices):
        # permute predictions following indices
        batch_idx = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
        src_idx = torch.cat([src for (src, _) in indices])
        return batch_idx, src_idx

    def _get_tgt_permutation_idx(self, indices):
        # permute targets following indices
        batch_idx = torch.cat([torch.full_like(tgt, i) for i, (_, tgt) in enumerate(indices)])
        tgt_idx = torch.cat([tgt for (_, tgt) in indices])
        return batch_idx, tgt_idx

    def get_loss(self, loss, outputs, targets, indices, num_points, **kwargs):
        loss_map = {
            'labels': self.loss_labels,
            'points': self.loss_points,
        }
#03/21 debug        print("loss_map {}".format(loss_map))
        assert loss in loss_map, f'do you really want to compute {loss} loss?'
        return loss_map[loss](outputs, targets, indices, num_points, **kwargs)

    def forward(self, outputs, targets):
        """ This performs the loss computation.
        Parameters:
             outputs: dict of tensors, see the output specification of the model for the format
             targets: list of dicts, such that len(targets) == batch_size.
                      The expected keys in each dict depends on the losses applied, see each loss' doc
        """
        output1 = {'pred_logits': outputs['pred_logits'], 'pred_points': outputs['pred_points']}
        indices1 = self.matcher(output1, targets)

        num_points = sum(len(t["labels"]) for t in targets)

        num_points = torch.as_tensor([num_points], dtype=torch.float, device=next(iter(output1.values())).device)
        if is_dist_avail_and_initialized():
            torch.distributed.all_reduce(num_points)
        #
        num_boxes = torch.clamp(num_points / get_world_size(), min=1).item()

        losses = {}
        for loss in self.losses:
            losses.update(self.get_loss(loss, output1, targets, indices1, num_boxes))

        return losses

# create the P2PNet model
def build(args, training):
    # treats persons as a single class
    num_classes = label_type_count

    model = P2PNet(args.row, args.line)
    if not training:
        return model

    weight_dict = {'loss_ce': label_type_count, 'loss_points': args.point_loss_coef}
#    weight_dict = {'loss_ce': 1, 'loss_points': args.point_loss_coef}  2023/03/19 バグ取り
    losses = ['labels', 'points']
    matcher = build_matcher_crowd(args)
    criterion = SetCriterion_Crowd(num_classes, \
                                matcher=matcher, weight_dict=weight_dict, \
                                eos_coef=args.eos_coef, losses=losses)

#03/21 debug    print("criterion:")
#03/21 debug    print(criterion)
    return model, criterion
###
# create the P2PNet model for execution
def build_eval(args):
    # treats persons as a single class
    num_classes = label_type_count

    model = P2PNet(args.row, args.line)
    return model
'''

'\n############\n# p2pnet.py\n############\n\n# build model p2pNet.py\nimport torch\nimport torch.nn.functional as F\nfrom torch import nn\n\nfrom util.misc import (NestedTensor, nested_tensor_from_tensor_list,\n                       accuracy, get_world_size, interpolate,\n                       is_dist_avail_and_initialized)\n\n# from models.backbone import build_backbone\n# from models.matcher import build_matcher_crowd\n\nimport numpy as np\nimport time\n\n# the network frmawork of the regression branch\nclass RegressionModel(nn.Module):\n    def __init__(self, num_features_in, num_anchor_points=4, feature_size=32):\n        super(RegressionModel, self).__init__()\n\n        self.conv1 = nn.Conv2d(num_features_in, feature_size, kernel_size=3, padding=1)\n        self.act1 = nn.ReLU()\n\n        self.conv2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)\n        self.act2 = nn.ReLU()\n\n        self.output = nn.Conv2d(feature_size, num_anchor_points * 2, kernel_siz

In [12]:
def get_train_arguments():
    """Parse all the arguments provided from the CLI.
    Returns:
      A list of parsed arguments.
    """


    #Change into rice data !!! -> chage default value
    parser = argparse.ArgumentParser(description="Object Counting Framework")
    # constant
    parser.add_argument('--lr', default=learning_rate, type=float) # originall set 1e-3 and can be reduced at later training stage
    parser.add_argument('--lr_fpn', default=learning_rate_2, type=float)
    parser.add_argument('--batch_size', default=1, type=int)
#    parser.add_argument('--batch_size', default=4, type=int)

    parser.add_argument('--weight_decay', default=1e-4, type=float)
    parser.add_argument('--epochs', default=100, type=int)
    parser.add_argument('--lr_drop', default=2000000, type=int)
    parser.add_argument('--clip_max_norm', default=0.1, type=float, help='gradient clipping max norm')

    # Model parameters
    parser.add_argument('--frozen_weights', type=str, default=None, help="Path to the pretrained model. If set, only the mask head will be trained")

    # * Matcher is a Hungarian strategy, minimizing the cost of proposed points and gt points
    parser.add_argument('--set_cost_class', default=0.5, type=float,
                        help="Class coefficient in the matching cost") #クラスマッチングの失敗による重みづけ係数

    parser.add_argument('--set_cost_point', default=0.5, type=float,
                        help="L1 point coefficient in the matching cost")

    # * Loss coefficients
    parser.add_argument('--point_loss_coef', default=0.02, type=float) # default = 0.0002 #Change into rice data !!! -> chage default value
    parser.add_argument('--eos_coef', default=0.01, type=float, # default 0.05, should be set low at the beginning !
                        help="Relative classification weight of the no-object class") # default = 0.5#Change into rice data !!! -> chage default value

    # a threshold during evaluation for counting and visualization
    parser.add_argument('--threshold', default=0.5, type=float,
                        help="threshold in evalluation: evaluate_crowd_no_overlap")#Change into rice data !!! -> chage default value
    parser.add_argument('--row', default=2, type=int,
                        help="row number of anchor points")#Change into rice data !!! -> chage default value
    parser.add_argument('--line', default=2, type=int,
                        help="line number of anchor points")#Change into rice data !!! -> chage default value

    # dataset parameters
    parser.add_argument('--dataset_file', default='SHHA')
    parser.add_argument('--data_root', default=training_data_root_dir,
                        help='path where the dataset is')#Change into rice data !!! -> chage default value

    parser.add_argument('--output_dir', default=inference_output_path,
                        help='path where to save, empty for no saving')
    parser.add_argument('--checkpoints_dir', default = program_files_root_dir + 'CrowdCounting-P2PNet/MultiLevelPyramidFeature_01',
                        help='path where to save checkpoints, empty for no saving') #ckpt_5n was not bad, default 2 X 2#Change into rice data !!! -> chage default value
    parser.add_argument('--tensorboard_dir', default=program_files_root_dir + 'CrowdCounting-P2PNet/Grain_runs',
                        help='path where to save, empty for no saving')#Change into rice data !!! -> chage default value

    parser.add_argument('--seed', default=42, type=int)#Change into rice data !!! -> chage default value
    parser.add_argument('--resume', default='', help='resume from checkpoint')#Change into rice data !!! -> chage default value
    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')#Change into rice data !!! -> chage default value
    parser.add_argument('--eval', action='store_true')
    parser.add_argument('--num_workers', default=1, type=int)
    parser.add_argument('--eval_freq', default=5, type=int,
                        help='frequency of evaluation, default setting is evaluating in every 5 epoch')#Change into rice data !!! -> chage default value
    parser.add_argument('--gpu_id', default=0, type=int, help='the gpu used for training')
    #
    opt = parser.parse_known_args()[0] #if known else parser.parse_args()
    return opt

## Start training main()

In [13]:
train_args = get_train_arguments()
#
train_args.resume = ""

# added by myself.hwang
train_args.frozen_weights = None

# args.vis_dir = inference_output_path + "/Visualization_Soybean_MultiLevelPyramidFeature_out"
# train_args.vis_dir = inference_output_path + "/visual_pyramid"
train_args.vis_dir = vis_dir
# if not os.path.exists(train_args.vis_dir):
#     os.makedirs(train_args.vis_dir)

##ログファイルに記録する
os.environ["CUDA_VISIBLE_DEVICES"] = '{}'.format(train_args.gpu_id)

# create the logging file
train_args.output_dir = inference_output_path
if not os.path.exists(train_args.output_dir):
    os.makedirs(train_args.output_dir)

run_log_name = os.path.abspath( os.path.join(inference_output_path, 'train_args.txt') )
with open(run_log_name, "w") as log_file:
    log_file.write('Eval Log %s\n' % time.strftime("%c"))

# backup the arguments
print(train_args)
with open(run_log_name, "a") as log_file:
    log_file.write("{}".format(train_args))

train_args.checkpoints_dir = checkpoints_dir
best_model_file = os.path.join(train_args.checkpoints_dir, 'best_mae.pth')

Namespace(lr=0.001, lr_fpn=0.0001, batch_size=1, weight_decay=0.0001, epochs=100, lr_drop=2000000, clip_max_norm=0.1, frozen_weights=None, set_cost_class=0.5, set_cost_point=0.5, point_loss_coef=0.02, eos_coef=0.01, threshold=0.5, row=2, line=2, dataset_file='SHHA', data_root='/home/hwang/d/GoogleDrive/howcanoewang@gmail.com/SharedWithMe/GrainCounting/AnalysisData_small/', output_dir='./outputs/exp_v2', checkpoints_dir='/home/hwang/d/GoogleDrive/howcanoewang@gmail.com/SharedWithMe/GrainCounting/programs/CrowdCounting-P2PNet/MultiLevelPyramidFeature_01', tensorboard_dir='/home/hwang/d/GoogleDrive/howcanoewang@gmail.com/SharedWithMe/GrainCounting/programs/CrowdCounting-P2PNet/Grain_runs', seed=42, resume='', start_epoch=0, eval=False, num_workers=1, eval_freq=5, gpu_id=0, vis_dir='./outputs/exp_v2/visual_pyramid')


In [14]:
train_img_list = []
solid_path = '/content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/'

with open(training_list_path, 'r') as f:
    for line in f.readlines():
        # one line: `path/img_name.JPG,path/img_name.txt`
        split_content = line.split(',')

        new_split_content = [item.replace(solid_path, training_data_root_dir ).replace('\n', '') for item in split_content]

        train_img_list.append( new_split_content )

In [15]:
eval_img_list = []
solid_path = '/content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/'

with open(evaluating_list_path, 'r') as f:
    for line in f.readlines():
        # one line: `path/img_name.JPG,path/img_name.txt`
        split_content = line.split(',')

        new_split_content = [item.replace(solid_path, training_data_root_dir ).replace('\n', '') for item in split_content]

        eval_img_list.append( new_split_content )

In [16]:
device = torch.device('cuda')

# fix the seed for reproducibility
seed = train_args.seed + utils.get_rank()
seed = train_args.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
# get the P2PNet model
model, criterion = build(train_args, training=True)

# move to GPU
model.to(device)
criterion.to(device)

model_without_ddp = model

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)
# use different optimation params for different parts of the model
param_dicts = [
    {"params": [p for n, p in model_without_ddp.named_parameters() if "fpn" not in n and p.requires_grad]},
    {
        "params": [p for n, p in model_without_ddp.named_parameters() if "fpn" in n and p.requires_grad],
        "lr": train_args.lr_fpn,
    },
]
# Adam is used by default
optimizer = torch.optim.Adam(param_dicts, lr=train_args.lr)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, train_args.lr_drop)

# create the training and valiation set
# train_set, val_set = loading_data(args.data_root)  # args_dataroot = training_data_root_dir
train_set, val_set = loading_data( training_data_root_dir )

# create the sampler used during training
sampler_train = torch.utils.data.RandomSampler(train_set)
sampler_val = torch.utils.data.SequentialSampler(val_set)

batch_sampler_train = torch.utils.data.BatchSampler(
    sampler_train, train_args.batch_size, drop_last=True)
# the dataloader for training
data_loader_train = DataLoader(train_set, batch_sampler=batch_sampler_train,
                                collate_fn=utils.collate_fn_crowd, num_workers=train_args.num_workers)

data_loader_val = DataLoader(val_set, 1, sampler=sampler_val,
                                drop_last=False, collate_fn=utils.collate_fn_crowd, num_workers=train_args.num_workers)

if train_args.frozen_weights is not None:
    # checkpoint = torch.load(args.frozen_weights, map_location='cpu')
    checkpoint = torch.load(train_args.frozen_weights, weights_only=False, map_location=device)
    model_without_ddp.detr.load_state_dict(checkpoint['model'])

# resume the weights and training state if exists
if train_args.resume:
    print("resume")
    # checkpoint = torch.load(args.resume, map_location='cpu')
    checkpoint = torch.load(train_args.resume, weights_only=False, map_location=device)
    model_without_ddp.load_state_dict(checkpoint['model'], strict=False)
    if not train_args.eval and 'optimizer' in checkpoint and 'lr_scheduler' in checkpoint and 'epoch' in checkpoint:
      optimizer.load_state_dict(checkpoint['optimizer'])
      lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
      train_args.start_epoch = checkpoint['epoch'] + 1
      print(checkpoint['epoch'] + 1)

number of params: 15053426


Start training

In [17]:
print("Start training")
start_time = time.time()

# the logger writer
writer = SummaryWriter(train_args.tensorboard_dir)


# save the performance during the training
mae = []
mse = []

# threshold for evaluation
threshold = train_args.threshold
#
step = 0
# training starts here
for epoch in range(train_args.start_epoch, train_args.epochs):

    # run evaluation
    if epoch % train_args.eval_freq == 0:
        t1 = time.time()
        result = evaluate_crowd_no_overlap(model, data_loader_val, device, class_n=2, threshold=threshold)
        t2 = time.time()


        # save the best model since begining
        if epoch > 0:
            if np.min(mse) > result[1]:
            # if abs(np.min(mae) - result[0]) < 0.01:
                checkpoint_best_path = best_model_file
                torch.save({
                    'model': model_without_ddp.state_dict(),
                }, checkpoint_best_path)
                print("===updated best model===")

        mae.append(result[0])
        mse.append(result[1])

        print(mae)
        print(mse)

        # print the evaluation results
        print('=======================================test=======================================')
#          print("mae:", result[0], "mse:", result[1], "time:", t2 - t1, "best mae:", np.min(mae), )
        print("mae:", result[0], "mse:", result[1], "time:", t2 - t1, "best mse:", np.min(mse), )
        with open(run_log_name, "a") as log_file:
#              log_file.write("mae:{}, mse:{}, time:{}, best mae:{}".format(result[0],result[1], t2 - t1, np.min(mae)))
            log_file.write("mae:{}, mse:{}, time:{}, best mse:{}".format(result[0],result[1], t2 - t1, np.min(mse)))
        print('=======================================test=======================================')
        # recored the evaluation results
        if writer is not None:
            with open(run_log_name, "a") as log_file:
                log_file.write("metric/mae@{}: {}".format(step, result[0]))
                log_file.write("metric/mse@{}: {}".format(step, result[1]))
            writer.add_scalar('metric/mae', result[0], step)
            writer.add_scalar('metric/mse', result[1], step)
            step += 1



    t1 = time.time()
    stat = train_one_epoch(model, criterion, data_loader_train, optimizer, device, epoch, train_args.clip_max_norm)

    # record the training states after every epoch
    if writer is not None:
        with open(run_log_name, "a") as log_file:
            log_file.write("loss/loss@{}: {}".format(epoch, stat['loss']))
            log_file.write("loss/loss_ce@{}: {}".format(epoch, stat['loss_ce']))

        writer.add_scalar('loss/loss', stat['loss'], epoch)
        writer.add_scalar('loss/loss_ce', stat['loss_ce'], epoch)

    t2 = time.time()
    print('[ep %d][lr %.7f][%.2fs]' % \
            (epoch, optimizer.param_groups[0]['lr'], t2 - t1))
    with open(run_log_name, "a") as log_file:
        log_file.write('[ep %d][lr %.7f][%.2fs]' % (epoch, optimizer.param_groups[0]['lr'], t2 - t1))
    # change lr according to the scheduler
    lr_scheduler.step()
    # save latest weights every epoch
    if not os.path.exists(train_args.checkpoints_dir):
        os.makedirs(train_args.checkpoints_dir)
    checkpoint_latest_path = os.path.join(train_args.checkpoints_dir, 'latest.pth')
    torch.save({
        'model': model_without_ddp.state_dict(),
    }, checkpoint_latest_path)
    if epoch % 300 == 0 and epoch != 0:
        clear_output()
# total time for training
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

Start training
[np.float64(5.4103525881470365)]
[np.float64(5.641981743102097)]
=======================================test=======================================
mae: 5.4103525881470365 mse: 5.641981743102097 time: 3.422203302383423 best mse: 5.641981743102097
=======================================test=======================================
Averaged stats: lr: 0.001000  loss: 0.1790 (0.3241)  loss_ce: 0.1730 (0.3002)  loss_points: 0.0113 (0.0239)  loss_ce_unscaled: 0.0577 (0.1001)  loss_points_unscaled: 0.5647 (1.1973)
[ep 0][lr 0.0010000][26.27s]
Averaged stats: lr: 0.001000  loss: 0.1288 (0.2541)  loss_ce: 0.1074 (0.2248)  loss_points: 0.0158 (0.0294)  loss_ce_unscaled: 0.0358 (0.0749)  loss_points_unscaled: 0.7913 (1.4680)
[ep 1][lr 0.0010000][25.89s]
Averaged stats: lr: 0.001000  loss: 0.0869 (0.2199)  loss_ce: 0.0683 (0.2038)  loss_points: 0.0141 (0.0160)  loss_ce_unscaled: 0.0228 (0.0679)  loss_points_unscaled: 0.7066 (0.8023)
[ep 2][lr 0.0010000][25.97s]
Averaged stats: lr: 0.

KeyboardInterrupt: 